In [111]:
import pandas as pd


movies_data_file_name = 'movies_metadata.csv'
credits_data_file_name = 'credits.csv'

def load_data(data_file_name):
    return pd.read_csv(data_file_name, encodeing='latin-1')


movies_data = load_data(movies_data_file_name)
credits_data = load_data(credits_data_file_name)


TypeError: read_csv() got an unexpected keyword argument 'encodeing'

In [ ]:
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="dbeaver",
  password="dbeaver",
  database="project"
)
mycursor = mydb.cursor()


In [ ]:
class Column:
    def __init__(self, name: str, _type: str, converter_funtion = None):
        self.name = name
        self.type = _type
        self.converter_funtion = converter_funtion


class Table:
    def __init__(self, mycursor, name: str, data: pd.DataFrame, columns: list[Column]):
        self.name = name
        self.data = data
        self.columns = columns
        self._mycursor = mycursor
    
    def get_records(self):
        records = []
        for column in self.columns:
            records.append(self.data[column.name].apply(column.converter_funtion) if column.converter_funtion else self.data[column.name])
        return pd.DataFrame(records).T
    
    def get_columns(self):
        return self.columns

    def cretae_table_command(self):
        cmd = f"CREATE TABLE IF NOT EXISTS {self.name} ("
        for column in self.columns:
            cmd += f"{column.name} {column.type},"
        cmd = cmd[:-1]
        cmd += ")"
        self._mycursor.execute(cmd)

    def insert_records(self):
        cmd = f"INSERT INTO {self.name} ("
        for column in self.columns:
            cmd += f"{column.name},"
        cmd = cmd[:-1]
        cmd += ") VALUES ("
        for i in range(len(self.columns)):
            cmd += "%s,"
        cmd = cmd[:-1]
        cmd += ")"
        self._mycursor.executemany(cmd, self.get_records().values.tolist())


movies_table = Table(
    mycursor,
    "movies",
    movies_data,
    [
        Column("imdb_id", "INT", lambda x: int(x.replace('tt', '')) if x and type(x) == str else None),
        Column("title", "VARCHAR(255)")]
)
print(movies_table.cretae_table_command())
print(movies_table.insert_records())

None
INSERT INTO movies (imdb_id,title) VALUES (%s,%s)
None


In [ ]:
mycursor.execute("SHOW TABLES")

for x in mycursor:
  print(x)



In [ ]:
credits_data = load_data(credits_data_file_name)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xfd in position 50974: invalid start byte

In [ ]:
import json

def load_data(data_file_name):
    return pd.read_csv(data_file_name, encodeing='latin-1')


def get_crew(credits_data):
    data = []

    for i in range(len(credits_data)):
        crew = credits_data['crew'][i]
        crew = crew.replace('\'', '"')
        crew = crew.replace('[', '')
        crew = crew.replace(']', '')
        for c in crew.split('},'):
            try:
                crew = json.loads(c + '}')
                data.append([credits_data['id'][i], crew['name'], crew['job']]) 
            except:
                pass
    crew_to_movie = pd.DataFrame(data, columns=['id', 'name', 'job'])
    crew_to_movie = crew_to_movie.sort_values(by=['id'])
    crew_to_movie = crew_to_movie.dropna()
    crew_to_movie = crew_to_movie[crew_to_movie['id'] <= 400]
    
    crew = crew_to_movie.copy()
    crew = crew.drop_duplicates(subset='name')
    crew = crew.reset_index(drop=True)
    crew['id'] = crew.index + 1
    crew.to_csv('crew_members.csv', index=False)

    crew_to_movie = crew_to_movie.merge(crew, on='name')
    crew_to_movie = crew_to_movie.drop(columns=['name'])
    crew_to_movie = crew_to_movie.drop(columns=['job_x'])
    crew_to_movie = crew_to_movie.rename(columns={'id_x': 'movie_id', 'id_y': 'crew_id', 'job_y': 'role'})
    crew_to_movie.to_csv('crew_members_movies.csv', index=False)

def get_cast():
    data = []

    for i in range(len(credits_data)):
        cast = credits_data['cast'][i]
        cast = cast.replace('\'', '"')
        cast = cast.replace('[', '')
        cast = cast.replace(']', '')
        for c in cast.split('},'):
            try:
                cast = json.loads(c + '}')
                data.append([credits_data['id'][i], cast['name']]) 
            except:
                pass
    cast_to_movie = pd.DataFrame(data, columns=['id', 'name'])
    cast_to_movie = cast_to_movie.sort_values(by=['id'])
    cast_to_movie = cast_to_movie.dropna()
    cast_to_movie = cast_to_movie[cast_to_movie['id'] <= 400]

    # keep only names unique and add id
    cast_data = cast_to_movie.copy()
    cast_data = cast_data.drop_duplicates(subset='name')
    cast_data = cast_data.reset_index(drop=True)
    cast_data['id'] = cast_data.index + 1
    cast_data.to_csv('actors.csv', index=False)

    # replace in cast_to_movie the name with id
    cast_to_movie = cast_to_movie.merge(cast_data, on='name')
    cast_to_movie = cast_to_movie.drop(columns=['name'])
    cast_to_movie = cast_to_movie.rename(columns={'id_x': 'movie_id', 'id_y': 'actor_id'})
   
    cast_to_movie.to_csv('actor_movies.csv', index=False)


get_cast()
get_crew(credits_data)

In [114]:
import pandas as pd
def load_data(data_file_name):
    return pd.read_csv(data_file_name)

am = load_data('crew_members_movies.csv')
# sort and drop duplicates
am = am.sort_values(by=['movie_id', 'crew_id'])
am = am.drop_duplicates(subset=['movie_id', 'crew_id'])
am.to_csv('crew_members_movies.csv', index=False)
